In [11]:
import pandas as pd

In [12]:
EXCEL_PATH = "/Users/janehrlinspiel/Library/CloudStorage/OneDrive-Persönlich/Crypto/Daten/crypto_7years_daily.xlsx"

In [13]:
import pandas as pd
from backtester import Backtester
from strategies.trend_following import TrendFollowingStrategy
from strategies.mean_reversion import MeanReversionStrategy

strategy_class = MeanReversionStrategy
param_grid = {
    "window": [10, 20, 50],
}
initial_cash = 100000

# Load all sheet names (one per asset)
xls = pd.read_excel(EXCEL_PATH,sheet_name="Cleaned data", parse_dates=["DATES"])
xls.drop_duplicates(inplace=True)
xls.set_index("DATES", inplace=True)

from itertools import product

def get_param_combinations(grid):
    keys = list(grid.keys())
    values = list(grid.values())
    for combo in product(*values):
        yield dict(zip(keys, combo))

results = []


In [14]:
xls.head()

,Enjin Coin,Maker,Solana,Cardano,Monero,Litecoin,Dogecoin,Chainlink,Bitcoin,XRP,...,Tether,Dai,USD Coin,Basic Attention Token,Cosmos,Cosmos.1,TerraUSD,Decentraland,Shiba Inu,Aave
DATES,,,,,,,,,,,,,,,,,,,,,
2018-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-11,NaN,NaN,NaN,NaN,NaN,189.485,NaN,NaN,9613.95,0.8200,...,0.9998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-12,NaN,NaN,NaN,NaN,NaN,178.825,NaN,NaN,9017.65,0.7819,...,0.9989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-13,NaN,NaN,NaN,NaN,NaN,174.265,NaN,NaN,9062.92,0.7697,...,1.0016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-03-14,NaN,NaN,NaN,NaN,NaN,162.745,NaN,NaN,8304.32,0.6988,...,0.9983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#get columns names in the dataframe
assets = xls.columns.tolist()

In [16]:


for asset in assets:
    df = xls[[asset]]
    for params in get_param_combinations(param_grid):
        strategy = strategy_class(df.columns[0],**params)

        bt = Backtester(
            data=df.copy(),  # changed from file path to raw data
            strategy=strategy,
            initial_cash=initial_cash
        )
        bt.run()
        bt.evaluate(silent=True)

        result = {
            "asset": asset,
            **params,
            **bt.get_performance_metrics()  # e.g., return, sharpe
        }

        results.append(result)

# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df.sort_values(by="total_return", ascending=False, inplace=True)
results_df.reset_index(drop=True, inplace=True)
print(results_df)


             asset  window  total_return  annualized_return  sharpe
0       Enjin Coin      10      525.9726             1.0189    1.12
1     Binance Coin      10        6.9304             0.2815    0.76
2             IOTA      10        6.5766             0.2541    0.68
3             IOTA      20        5.5528             0.2339    0.64
4       Enjin Coin      50        1.5877             0.1125    0.58
..             ...     ...           ...                ...     ...
145       Filecoin      20       -0.9941            -0.5971   -1.05
146          Zcash      20       -0.9972            -0.4595   -0.69
147  Terra Classic      50       -1.0013                NaN   -0.35
148  Terra Classic      20       -1.0165                NaN   -0.27
149     Enjin Coin      20      -14.0327                NaN    0.87

[150 rows x 5 columns]


/Users/janehrlinspiel/Library/CloudStorage/OneDrive-Persönlich/Crypto/Code/Backtester_git/backtester.py:45: RuntimeWarning: invalid value encountered in scalar power
  annualized_return = (1 + total_return) ** (252 / len(self.results)) - 1
/Users/janehrlinspiel/Library/CloudStorage/OneDrive-Persönlich/Crypto/Code/Backtester_git/backtester.py:45: RuntimeWarning: invalid value encountered in scalar power
  annualized_return = (1 + total_return) ** (252 / len(self.results)) - 1
/Users/janehrlinspiel/Library/CloudStorage/OneDrive-Persönlich/Crypto/Code/Backtester_git/backtester.py:45: RuntimeWarning: invalid value encountered in scalar power
  annualized_return = (1 + total_return) ** (252 / len(self.results)) - 1


In [17]:
results_df[results_df["asset"] == 'Bitcoin']

,asset,window,total_return,annualized_return,sharpe
46,Bitcoin,10,-0.5595,-0.0805,-0.07
58,Bitcoin,20,-0.6943,-0.1143,-0.14
64,Bitcoin,50,-0.7707,-0.1400,-0.21
